In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import datetime as dt

# 1. Performance Metrics

In [2]:
equities = pd.read_excel("../data/industry_equity_data.xlsx", sheet_name = 'total returns')
equities.set_index('Date',inplace=True)

In [3]:
def performanceMetrics(returns):
    metrics = pd.DataFrame(index=returns.columns)
    metrics['Mean'] = returns.mean()  * 12
    metrics['Vol'] = returns.std() * np.sqrt(12)
    metrics['Sharpe'] = (returns.mean() / returns.std()) * np.sqrt(12)

    metrics['Min'] = returns.min()
    metrics['Max'] = returns.max()
    
    metrics['VaR (5th)'] = returns.quantile(.05)
    metrics['CVaR (5th)'] = (returns[returns < returns.quantile(.05)]).mean()

    return metrics

In [4]:
metrics = performanceMetrics(equities)
metrics.round(4)

,Mean,Vol,Sharpe,Min,Max,VaR (5th),CVaR (5th)
AAPL,0.3272,0.3994,0.8192,-0.5774,0.4538,-0.1451,-0.2495
AMZN,0.2821,0.4591,0.6145,-0.4116,0.6218,-0.1778,-0.2775
GS,0.1272,0.3215,0.3956,-0.2751,0.3122,-0.1458,-0.1849
JPM,0.1266,0.3021,0.4190,-0.2807,0.2555,-0.1412,-0.2022
MSFT,0.1334,0.2850,0.4680,-0.3435,0.4078,-0.1207,-0.1678
SPY,0.0797,0.1504,0.5302,-0.1652,0.1270,-0.0753,-0.0973
XLB,0.0968,0.2059,0.4701,-0.2240,0.1734,-0.0924,-0.1289
XLE,0.0867,0.2487,0.3487,-0.3437,0.3076,-0.1114,-0.1554
XLF,0.0738,0.2162,0.3415,-0.2620,0.2179,-0.0926,-0.1537
XLI,0.0938,0.1881,0.4987,-0.1863,0.1807,-0.0915,-0.1281


In [5]:
def maximumDrawdown(returns):
    cum_returns = (1 + returns).cumprod()
    rolling_max = cum_returns.cummax()
    drawdown = (cum_returns - rolling_max) / rolling_max

    max_drawdown = drawdown.min()
    end_date = drawdown.idxmin()    
    summary = pd.DataFrame({'Max Drawdown':max_drawdown, 'Bottom':end_date})
    
    
    # The rest of this code is to get the peak and Recover dates.
    # It is tedious, and I recommend skipping the rest of this code unless you are 
    # already comfortable with Python and Pandas.
    
    # get the date at which the return recovers to previous high after the drawdown
    summary['Recover'] = None
    for col in returns.columns:
        idx = returns.index[(returns.index >= end_date[col]).argmax()]
        check_recover = (cum_returns.loc[idx:,col] > rolling_max.loc[idx,col])
        if check_recover.any():                
            summary.loc[col,'Recover'] = check_recover.idxmax()            
    summary['Recover'] = pd.to_datetime(summary['Recover'])

    
    # get the date at which the return peaks before entering the max drawdown
    summary.insert(loc=1, column='Peak',value=0)
    for col in returns.columns:
        df = rolling_max.copy()[[col]]
        df.columns=['max']
        df['max date'] = df.index
        df = df.merge(df.groupby('max')[['max date']].first().reset_index(), on='max')
        df.rename(columns={'max date_y': 'max date', 'max date_x': 'date'}, inplace=True)
        df.set_index('date',inplace=True)

        summary.loc[col,'Peak'] = df.loc[end_date[col],'max date']

    summary['Peak'] = pd.to_datetime(summary['Peak'])
    summary['Peak to Recover'] = (summary['Recover'] - summary['Peak'])
        
    return summary

In [6]:
max_drawdown = maximumDrawdown(equities)
max_drawdown.round(4)

,Max Drawdown,Peak,Bottom,Recover,Peak to Recover
AAPL,-0.7918,2000-03-31,2003-03-31,2005-01-31,1767 days
AMZN,-0.9133,2000-02-29,2001-09-30,2007-05-31,2648 days
GS,-0.6786,2007-10-31,2008-11-30,2016-12-31,3349 days
JPM,-0.6455,2000-03-31,2002-09-30,2006-07-31,2313 days
MSFT,-0.6341,2000-03-31,2009-02-28,2014-03-31,5113 days
SPY,-0.5078,2007-10-31,2009-02-28,2012-03-31,1613 days
XLB,-0.5549,2008-05-31,2009-02-28,2013-04-30,1795 days
XLE,-0.6397,2014-06-30,2020-03-31,NaT,NaT
XLF,-0.7864,2007-05-31,2009-02-28,2017-07-31,3714 days
XLI,-0.5715,2007-09-30,2009-02-28,2011-04-30,1308 days


## OLS Regression

Here we use the package sci-kit learn for the OLS regression.
Many of you use statsmodels, which we used in the demo.

Both packages are widely used for OLS regression. Feel free to use either, but it is good to get exposure to both.

In [7]:
def get_ols_metrics(regressors,targets,annualization=1):
    
    # ensure regressors and targets are pandas dataframes, as expected
    if not isinstance(regressors,pd.DataFrame):
        regressors = regressors.to_frame()
    if not isinstance(targets,pd.DataFrame):
        targets = targets.to_frame()

    # align the targets and regressors on the same dates
    df_aligned = targets.join(regressors,how='inner',lsuffix='y ')
    Y = df_aligned[targets.columns]
    X = df_aligned[regressors.columns]

    reg = pd.DataFrame(index=targets.columns)
    for col in Y.columns:
        y = Y[col]
        model = LinearRegression().fit(X,y)
        reg.loc[col,'alpha'] = model.intercept_ * annualization
        reg.loc[col,regressors.columns] = model.coef_
        reg.loc[col,'r-squared'] = model.score(X,y)

        # sklearn does not return the residuals, so we need to build them
        yfit = model.predict(X)
        residuals = y - yfit    
        reg.loc[col,'Info Ratio'] = (model.intercept_ / residuals.std()) * np.sqrt(annualization)

    return reg

In [8]:
metrics_mkt = get_ols_metrics(equities['SPY'],equities,annualization=12)
metrics_mkt['Treynor'] = metrics['Mean'] / metrics_mkt['SPY']
metrics_mkt.round(4)

,alpha,SPY,r-squared,Info Ratio,Treynor
AAPL,0.2143,1.4155,0.2842,0.6342,0.2311
AMZN,0.1574,1.5638,0.2625,0.3992,0.1804
GS,0.0097,1.4728,0.4749,0.0417,0.0863
JPM,0.0172,1.3712,0.4662,0.0780,0.0923
MSFT,0.0430,1.1330,0.3575,0.1883,0.1177
SPY,0.0000,1.0000,1.0000,0.4597,0.0797
XLB,0.0058,1.1407,0.6947,0.0510,0.0848
XLE,-0.0015,1.1067,0.4482,-0.0084,0.0783
XLF,-0.0221,1.2028,0.7004,-0.1867,0.0614
XLI,0.0029,1.1396,0.8305,0.0378,0.0823


# 2. Return Decomposition

In [9]:
idx = equities.columns.str.startswith('X')
industry_etfs = equities.loc[:,idx]
single_names = equities.drop(columns=list(industry_etfs.columns) + ['SPY'])

get_ols_metrics(industry_etfs,single_names,annualization=12)

,alpha,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,r-squared,Info Ratio
AAPL,0.249835,0.208433,0.144431,-0.061541,-0.560630,1.445877,-0.176359,0.091021,-0.251767,0.068801,0.505519,0.889602
AMZN,0.143849,0.049241,-0.152409,-0.675339,0.290487,0.941006,-0.427819,-0.121019,0.753430,0.597603,0.439359,0.418494
GS,0.066471,0.408044,0.197433,0.881023,-0.352721,0.596822,-0.123809,-0.227468,-0.048251,-0.406789,0.603932,0.328558
JPM,0.072540,0.187051,-0.023690,1.353622,-0.436796,0.341595,-0.202190,-0.174183,0.097386,-0.249027,0.708363,0.444701
MSFT,0.057941,0.058496,-0.046859,0.125182,-0.339097,0.915551,-0.129999,0.059792,0.169335,0.076024,0.522413,0.294137


# 3. Fama-French Factors

In [10]:
ff = pd.read_excel("../data/fama_french_data.xlsx", sheet_name = 'total returns')
ff.set_index(ff.columns[0],inplace=True)
ff.index.name = 'Date'
ff.head()

,MKT,SMB,HML,UMD,RF
Date,,,,,
2000-01-31,-0.0433,0.0611,-0.0134,0.0227,0.0041
2000-02-29,0.0288,0.2162,-0.0851,0.1879,0.0043
2000-03-31,0.0567,-0.1635,0.0813,-0.0592,0.0047
2000-04-30,-0.0594,-0.0647,0.0787,-0.0811,0.0046
2000-05-31,-0.0392,-0.0583,0.0551,-0.0858,0.0050


In [11]:
ff_excess = ff[['MKT','SMB','HML','UMD']].subtract(ff['RF'],axis=0)
ff_excess.head()

,MKT,SMB,HML,UMD
Date,,,,
2000-01-31,-0.0474,0.0570,-0.0175,0.0186
2000-02-29,0.0245,0.2119,-0.0894,0.1836
2000-03-31,0.0520,-0.1682,0.0766,-0.0639
2000-04-30,-0.0640,-0.0693,0.0741,-0.0857
2000-05-31,-0.0442,-0.0633,0.0501,-0.0908


In [12]:
performanceMetrics(ff_excess).round(4)

,Mean,Vol,Sharpe,Min,Max,VaR (5th),CVaR (5th)
MKT,0.0697,0.1572,0.4430,-0.1723,0.1365,-0.0794,-0.1021
SMB,0.0306,0.1124,0.2721,-0.1682,0.2119,-0.0401,-0.0591
HML,0.0200,0.1149,0.1742,-0.1396,0.1258,-0.0427,-0.0724
UMD,0.0181,0.1839,0.0983,-0.3439,0.1836,-0.0844,-0.1354


## Fama French Regressions

In [13]:
targets_excess = equities[['AAPL','GS']].subtract(ff['RF'],axis=0)
get_ols_metrics(ff_excess,targets_excess,annualization=12)

,alpha,MKT,SMB,HML,UMD,r-squared,Info Ratio
AAPL,0.229111,1.352679,0.117844,-0.909914,0.001293,0.362616,0.717130
GS,0.003231,1.440958,0.168063,0.184939,-0.009701,0.527299,0.014584
